**Bert model op basis van Bert van Universiteit van Groningen. Context handling moet nog aangepast worden. Oversampling a.d.h.v de mediaan. dynamisch treshhold zoeken voor unknown.**

In [1]:
import pandas as pd
import re
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from imblearn.over_sampling import RandomOverSampler
import torch.nn.functional as F
import os 

c:\Users\corne\anaconda3\envs\thesis-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ✅ 1. Load & Preprocess Data
script_dir = os.path.dirname(os.getcwd()) # Ga één map omhoog om 'baseline' te verwijderen en ga naar 'Data'
project_root = os.path.dirname(script_dir)  # Dit verwijdert 'baseline' van het script_dir
data_folder = os.path.join(project_root, "Data")

# 1. Dataset inladen
file_path = os.path.join(data_folder, "Grote_data_cleaned.xlsx")
df = pd.read_excel(file_path)

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102013 entries, 0 to 102012
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   context      101403 non-null  object
 1   question     102013 non-null  object
 2   statistical  102013 non-null  int64 
 3   theme        102013 non-null  object
 4   file_name    102013 non-null  object
 5   clean_text   102013 non-null  object
dtypes: int64(1), object(5)
memory usage: 4.7+ MB


,context,question,statistical,theme,file_name,clean_text
0,Ondertussen is de eerstelijnszone BruZEL al me...,Zoals alle eerstelijnszones kreeg ook BruZEL h...,0,Brussel en de Vlaamse Rand,1752898.txt,Zoals alle eerstelijnszones kreeg ook BruZEL h...
1,Ondertussen is de eerstelijnszone BruZEL al me...,a)Wat liep er moeilijk?,0,Brussel en de Vlaamse Rand,1752898.txt,Wat liep er moeilijk?
2,Ondertussen is de eerstelijnszone BruZEL al me...,Met welke uitdagingen werd BruZEL het afgelo...,0,Brussel en de Vlaamse Rand,1752898.txt,Met welke uitdagingen werd BruZEL het afgelope...
3,Ondertussen is de eerstelijnszone BruZEL al me...,Hoe kunnen die uitdagingen worden aangepakt?,0,Brussel en de Vlaamse Rand,1752898.txt,Hoe kunnen die uitdagingen worden aangepakt?
4,Ondertussen is de eerstelijnszone BruZEL al me...,b)Wat liep er goed?,0,Brussel en de Vlaamse Rand,1752898.txt,Wat liep er goed?


In [ ]:
# ✅ Recompute label encoding AFTER filtering
unique_themes = list(df["theme"].unique())
theme_to_id = {theme: idx for idx, theme in enumerate(unique_themes)}
id_to_theme = {idx: theme for theme, idx in theme_to_id.items()}
df["theme_id"] = df["theme"].map(theme_to_id)

Number of rows after filtering: 102013


In [6]:
# ✅ 5. Split Data into Train & Test
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["clean_text"].tolist(), df["theme_id"].tolist(), test_size=0.2, random_state=42, stratify=df["theme_id"]
)

In [7]:
from imblearn.over_sampling import RandomOverSampler
import pandas as pd

# Create a small DataFrame from train lists
train_df = pd.DataFrame({
    "clean_text": train_texts,
    "theme_id": train_labels
})

# Compute class counts and use median as balancing target
theme_counts = train_df["theme_id"].value_counts()
median_count = theme_counts.median()

# Define strategy: only oversample underrepresented classes
sampling_strategy = {
    theme: int(median_count)
    for theme in theme_counts.index
    if theme_counts[theme] < median_count
}

# Apply RandomOverSampler
ros = RandomOverSampler(sampling_strategy=sampling_strategy, random_state=42)
X_resampled, y_resampled = ros.fit_resample(train_df[["clean_text"]], train_df["theme_id"])

# Extract oversampled train lists
train_texts_resampled = X_resampled["clean_text"].tolist()
train_labels_resampled = y_resampled.tolist()

from collections import Counter
print("Class distribution after oversampling:", Counter(train_labels_resampled))


Class distribution after oversampling: Counter({10: 20841, 2: 6668, 15: 6033, 12: 5419, 5: 4314, 27: 3635, 4: 3125, 1: 2926, 7: 2761, 14: 2646, 21: 2512, 11: 2416, 17: 1980, 29: 1621, 23: 1445, 16: 1395, 3: 1207, 13: 1171, 33: 1165, 8: 1135, 31: 1135, 9: 1135, 6: 1135, 36: 1135, 24: 1135, 22: 1135, 25: 1135, 19: 1135, 34: 1135, 30: 1135, 32: 1135, 26: 1135, 0: 1135, 28: 1135, 20: 1135, 35: 1135, 18: 1135, 37: 1135})


In [8]:
# ✅ 7. Load BERT Tokenizer & Define Dataset Class
model_name = "GroNLP/bert-base-dutch-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)

class ThemeDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx], padding="max_length", truncation=True, max_length=self.max_length, return_tensors="pt"
        )
        encoding["labels"] = torch.tensor(self.labels[idx])
        return {key: val.squeeze(0) for key, val in encoding.items()}

train_dataset = ThemeDataset(train_texts_resampled, train_labels_resampled, tokenizer)
test_dataset = ThemeDataset(test_texts, test_labels, tokenizer)

In [8]:
# ✅ 8. Load BERT Model for Classification
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=df["theme_id"].nunique())

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at GroNLP/bert-base-dutch-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# ✅ 9. Define Training Arguments (With Early Stopping)
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,                 # 👈 Keep only the last checkpoint
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=8,  
    weight_decay=0.01,
    logging_dir="./logs",
    logging_strategy="epoch",  # 🔥 Log only once per epoch
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True
)

# ✅ 10. Define Metrics for Evaluation
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted", zero_division=1)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# ✅ 11. Train Model with Early Stopping
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # Stop if no improvement for 2 epochs
)

trainer.train()

c:\Users\corne\anaconda3\envs\thesis-env\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
 12%|█▎        | 3996/31968 [14:52<1:34:21,  4.94it/s]

{'loss': 1.9471, 'grad_norm': 28.5859432220459, 'learning_rate': 1.7501876876876876e-05, 'epoch': 1.0}


                                                      
 12%|█▎        | 3996/31968 [15:44<1:34:21,  4.94it/s]

{'eval_loss': 1.4470617771148682, 'eval_accuracy': 0.6287022463871799, 'eval_precision': 0.6432422258338969, 'eval_recall': 0.6287022463871799, 'eval_f1': 0.6159595317151576, 'eval_runtime': 51.978, 'eval_samples_per_second': 134.461, 'eval_steps_per_second': 16.815, 'epoch': 1.0}


 25%|██▌       | 7992/31968 [30:18<1:20:40,  4.95it/s]  

...

                                                       
100%|██████████| 31968/31968 [2:00:30<00:00,  5.65it/s]

{'eval_loss': 2.038515090942383, 'eval_accuracy': 0.7288596365717556, 'eval_precision': 0.7289905674585534, 'eval_recall': 0.7288596365717556, 'eval_f1': 0.7267558639213636, 'eval_runtime': 40.9373, 'eval_samples_per_second': 170.724, 'eval_steps_per_second': 21.35, 'epoch': 8.0}


100%|██████████| 31968/31968 [2:00:31<00:00,  4.42it/s]

{'train_runtime': 7231.648, 'train_samples_per_second': 35.36, 'train_steps_per_second': 4.421, 'train_loss': 0.5429843137930105, 'epoch': 8.0}


TrainOutput(global_step=31968, training_loss=0.5429843137930105, metrics={'train_runtime': 7231.648, 'train_samples_per_second': 35.36, 'train_steps_per_second': 4.421, 'total_flos': 6.73024012786729e+16, 'train_loss': 0.5429843137930105, 'epoch': 8.0})

SAVE MODEL TO KUL DRIVE

In [11]:
import os
from datetime import datetime
import numpy as np
import pandas as pd
import json


# === Create timestamped save path in OneDrive ===
run_id = datetime.now().strftime("%Y-%m-%d_%H-%M")
base_path = "C:/Users/corne/OneDrive - KU Leuven/Thesis/Working Code/SAVED-Models/GroNLP"
save_path = os.path.join(base_path, f"Run_{run_id}")
os.makedirs(save_path, exist_ok=True)

# === Save model using Trainer ===
trainer.save_model(save_path)

# === Get predictions ===
preds_output = trainer.predict(test_dataset)
logits = preds_output.predictions
predictions = np.argmax(logits, axis=1)

# === Save predictions to CSV ===
output_df = pd.DataFrame({
    "text": test_texts,  # make sure test_texts is defined
    "true_label": test_labels,  # make sure test_labels is defined
    "predicted_label": predictions,
    "logits": logits.tolist()
})

csv_path = os.path.join(save_path, "test_predictions.csv")
output_df.to_csv(csv_path, index=False)

# === Optional: Save a description file ===
description = """
Model: GroNLP BERT-based Dutch Cased
Training Details:
geen context meegegeven in zowel train als test
"""

desc_path = os.path.join(save_path, "model_description.txt")
with open(desc_path, "w", encoding="utf-8") as f:
    f.write(description)

# === Save label mappings ===
mappings_path = os.path.join(save_path, "label_mappings.json")
with open(mappings_path, "w", encoding="utf-8") as f:
    json.dump({
        "theme_to_id": theme_to_id,
        "id_to_theme": {str(k): v for k, v in id_to_theme.items()}  # keys must be str for JSON
    }, f, ensure_ascii=False, indent=4)


print(f"Everything saved in: {save_path}")


100%|██████████| 874/874 [00:40<00:00, 21.53it/s]


Everything saved in: C:/Users/corne/OneDrive - KU Leuven/Thesis/Working Code/SAVED-Models/GroNLP\Run_2025-04-10_15-17
